In [5]:
import pandas as pd
import numpy as np
from pydataset import data

np.random.seed(123)


In [2]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))


In [4]:
# Construct the DataFrame using the above lists and arrays.

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})


In [5]:
df

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,B
3,Billy,98,96,88,B
4,Ada,77,92,98,B
5,John,79,76,93,A
6,Thomas,82,64,81,A
7,Marie,93,63,90,A
8,Albert,92,62,87,A
9,Richard,69,80,94,B


In [6]:
pd.DataFrame([[1, 2, 3], [4, 5, 6]])


,0,1,2
0,1,2,3
1,4,5,6


In [7]:
array = np.array([[1, 2, 3], [4, 5, 6]])

pd.DataFrame(array, columns=['a', 'b', 'c'])


,a,b,c
0,1,2,3
1,4,5,6


## Exercises I

In [2]:
from env import host, user, password
url = f'mysql+pymysql://{user}:{password}@{host}/employees'

6. Read the employees and titles tables into two separate DataFrames.

In [6]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [8]:
pd.read_sql('SELECT * FROM titles LIMIT 5 OFFSET 50', url)

,emp_no,title,from_date,to_date
0,10036,Senior Staff,1992-04-28,9999-01-01
1,10037,Engineer,1990-12-05,1995-12-05
2,10037,Senior Engineer,1995-12-05,9999-01-01
3,10038,Senior Staff,1996-09-20,9999-01-01
4,10038,Staff,1989-09-20,1996-09-20


7. How many rows and columns do you have in each DataFrame? Is that what you expected?

Yes

8. Display the summary statistics for each DataFrame.

In [ ]:
describe()

9. How many unique titles are in the titles DataFrame?



In [12]:
query ='''
SELECT DISTINCT(title)
FROM titles
'''

unique_titles = pd.read_sql(query, url)
unique_titles

,title
0,Senior Engineer
1,Staff
2,Engineer
3,Senior Staff
4,Assistant Engineer
5,Technique Leader
6,Manager


10. What is the oldest date in the to_date column?

In [16]:
query2 ='''
SELECT to_date from titles
order by to_date ASC
limit 1'''

oldest_date = pd.read_sql(query2,url)
oldest_date

,to_date
0,1985-03-01


11. What is the most recent date in the to_date column?

In [17]:
query2 ='''
SELECT to_date FROM titles
WHERE to_date < curdate() 
ORDER BY to_date DESC
limit 1;'''

recent_date = pd.read_sql(query2,url)
recent_date

,to_date
0,2002-08-01


## Exercises II

In [8]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [9]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


2. What is the result of using a right join on the DataFrames?

In [11]:
users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


3. What is the result of using an outer join on the DataFrames?

In [12]:
users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [15]:
users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer',indicator=True).drop(columns='role_id')

,id_x,name_x,id_y,name_y,_merge
0,1.0,bob,1.0,admin,both
1,2.0,joe,2.0,author,both
2,3.0,sally,3.0,reviewer,both
3,4.0,adam,3.0,reviewer,both
4,5.0,jane,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,left_only
6,NaN,NaN,4.0,commenter,right_only


5. Load the mpg dataset from PyDataset.

In [21]:
mpg = data('mpg')

In [22]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [23]:
mpg.rename(columns= {'cty':'city'},inplace = True)
mpg.rename(columns= {'hwy':'highway'},inplace = True)

In [24]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [26]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   city          234 non-null    int64  
 8   highway       234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


In [27]:
mpg.describe()

,displ,year,cyl,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


10. How many different manufacturers are there?

In [35]:
unique_manu = mpg.manufacturer.unique()

In [45]:
unique_manu

array(['audi', 'chevrolet', 'dodge', 'ford', 'honda', 'hyundai', 'jeep',
       'land rover', 'lincoln', 'mercury', 'nissan', 'pontiac', 'subaru',
       'toyota', 'volkswagen'], dtype=object)

In [50]:
type(unique_manu)

numpy.ndarray

In [59]:
unique_manu.size

15

11. How many different models are there?

In [62]:
unique_model = mpg.model.unique()

In [64]:
unique_model.size

38

12. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [65]:
mpg['mileage_difference'] = mpg.highway - mpg.city

In [66]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


13. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [67]:
mean_highway_city = (mpg.highway + mpg.city)/2

In [68]:
mpg['average_mileage'] = mean_highway_city

14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [75]:
mpg['is_automatic'] = mpg['trans'].str.contains('auto')

In [76]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [83]:
mpg.sort_values(by = ['average_mileage'], ascending = False).head()

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference,average_mileage,is_automatic
222,volkswagen,new beetle,1.9,1999,4,manual(m5),f,35,44,d,subcompact,9,39.5,False
213,volkswagen,jetta,1.9,1999,4,manual(m5),f,33,44,d,compact,11,38.5,False
223,volkswagen,new beetle,1.9,1999,4,auto(l4),f,29,41,d,subcompact,12,35.0,True
197,toyota,corolla,1.8,2008,4,manual(m5),f,28,37,r,compact,9,32.5,False
196,toyota,corolla,1.8,1999,4,manual(m5),f,26,35,r,compact,9,30.5,False


16. Do automatic or manual cars have better miles per gallon?

In [ ]:
No 